In [1]:
import os
import sys
import numpy as np
import pandas as pd
from glob import glob
import rasterio as rio
import geopandas as gpd
from rasterio.plot import show
import matplotlib.pyplot as plt

# New Effective precip data for HUC12 from 2000-2022
USGS source: https://www.sciencebase.gov/catalog/item/6488734cd34ef77fcafe347a

In [2]:
# USGS effective precipitation unfiltered dataset for CONUS
conus_df = pd.read_csv('../../Data_main/USGS_water_use_data/USGS_new_wateruse_data_HUC12/Effective_precip_annual_huc12_2000_2020.csv')
conus_df

,Year,010100020101,010100020102,010100020103,010100020104,010100020105,010100020201,010100020202,010100020203,010100020204,...,181002041404,181002041501,181002041502,181002041503,181002041504,181002041505,181002041506,181002041507,181002041508,181002041600
0,2000,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,...,-0.836466,999.0,0.058759,0.042810,999.0,999.0,0.003947,0.122711,0.304217,0.010115
1,2001,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,...,-0.128781,999.0,0.190186,0.134202,999.0,999.0,0.013234,0.409960,0.959759,0.011789
2,2002,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,...,-0.285532,999.0,0.215754,0.222189,999.0,999.0,0.017996,0.487724,1.091170,0.009273
3,2003,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,...,0.116388,999.0,0.143022,0.301840,999.0,999.0,0.010656,0.363242,0.916381,0.012715
4,2004,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,...,0.080164,999.0,0.089734,-0.272009,999.0,999.0,0.009526,0.268748,0.616427,0.016310
5,2005,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,...,0.798718,999.0,0.268118,-0.024886,999.0,999.0,0.019434,0.550428,1.416297,0.045207
6,2006,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,...,0.393968,999.0,0.076435,-0.211628,999.0,999.0,0.007315,0.183015,0.451653,0.043392
7,2007,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,...,0.379975,999.0,0.029950,-0.183026,999.0,999.0,0.000896,0.044577,0.175021,0.021535
8,2008,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,...,0.083110,999.0,0.215697,0.269297,999.0,999.0,0.010755,0.322200,1.316652,0.073541
9,2009,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,...,-0.253225,999.0,0.054487,0.211769,999.0,999.0,0.003430,0.093200,0.318385,0.099327


In [4]:
# WestUS HUC12 shapefile
huc12_westUS = gpd.read_file('../../Data_main/shapefiles/HUCs/HUC12/HUC12WestUS.shp')
huc12_westUS.columns

Index(['noncontr00', 'sourcedata', 'areaacres', 'humod', 'sourcefeat', 'huc12',
       'shape_leng', 'tohuc', 'states', 'sourceorig', 'shape_area', 'areasqkm',
       'hutype', 'name', 'tnmid', 'metasource', 'loaddate', 'gnis_id',
       'noncontrib', 'geometry'],
      dtype='object')

In [5]:
# HUC12 no from west US HUC12 shapefile
westUS_HUC12s = list(huc12_westUS['huc12'])
westUS_HUC12s[:15]

['070200010408',
 '090201010203',
 '090201010502',
 '090201010505',
 '090201010507',
 '090201070101',
 '090201070104',
 '090201070106',
 '090201070601',
 '090201070603',
 '090201070605',
 '090203010401',
 '090203010402',
 '090203010404',
 '090201051006']

In [11]:
# Unique HUC12 no. in USGS database
usgs_HUC12s = [i for i in conus_df.columns if i not in ['Year']]
usgs_HUC12s[:10]

['010100020101',
 '010100020102',
 '010100020103',
 '010100020104',
 '010100020105',
 '010100020201',
 '010100020202',
 '010100020203',
 '010100020204',
 '010100020301']

In [12]:
# USGS HUC12 no. filtered with westUS HUC12 shapefile
usgs_HUC12s= set(usgs_HUC12s)
westUS_HUC12s = set(westUS_HUC12s)

usgs_HUC12s_filtered = list(usgs_HUC12s.intersection(westUS_HUC12s))
usgs_HUC12s_filtered[:10]

['110300010201',
 '170402150402',
 '170800050104',
 '170501070702',
 '181002040905',
 '150601010404',
 '110500020101',
 '170603080302',
 '120601040303',
 '170101011202']

In [13]:
# USGS HUC12 filtered for WestUS
usgs_HUC12s_filtered = ['Year'] + usgs_HUC12s_filtered
westUS_HUC12_df = conus_df[usgs_HUC12s_filtered]
westUS_HUC12_df[westUS_HUC12_df.isin([999, 888])] = 0   # setting null values to zero

westUS_HUC12_df.head()

C:\Users\fahim\AppData\Local\Temp\ipykernel_18456\1286272295.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  westUS_HUC12_df[westUS_HUC12_df.isin([999, 888])] = 0   # setting null values to zero
C:\Users\fahim\AppData\Local\Temp\ipykernel_18456\1286272295.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  westUS_HUC12_df[westUS_HUC12_df.isin([999, 888])] = 0   # setting null values to zero


,Year,110300010201,170402150402,170800050104,170501070702,181002040905,150601010404,110500020101,170603080302,120601040303,...,180600060203,100301010905,130500031903,160203040101,111302010607,180500021001,140401030401,150702030205,100800010903,140600031504
0,2000,0.005044,0.0,0.266710,0.0,0.0,0.0,0.207300,0.0,0.000000,...,0.0,0.009415,0.0,0.357999,0.0,0.0,0.0,0.0,0.0,0.193788
1,2001,0.031210,0.0,0.332165,0.0,0.0,0.0,0.001148,0.0,0.000000,...,0.0,0.050949,0.0,0.249631,0.0,0.0,0.0,0.0,0.0,0.169031
2,2002,0.044198,0.0,0.394226,0.0,0.0,0.0,0.351852,0.0,0.097313,...,0.0,0.000000,0.0,0.371579,0.0,0.0,0.0,0.0,0.0,0.160877
3,2003,0.133766,0.0,0.254837,0.0,0.0,0.0,0.554865,0.0,0.029803,...,0.0,0.005147,0.0,0.388419,0.0,0.0,0.0,0.0,0.0,0.216649
4,2004,0.049142,0.0,0.399572,0.0,0.0,0.0,0.921363,0.0,0.070826,...,0.0,0.037601,0.0,0.361747,0.0,0.0,0.0,0.0,0.0,0.199574


# Tranposing the columns as rows
The annual dataframe has HUC12 names as column names. But the WestUS HUC12 shapefile has HUC12 names in a column (as rows). We need to transpose the annual dataframe.

In [15]:
westUS_HUC12_df_T = westUS_HUC12_df.T
westUS_HUC12_df_T.columns = westUS_HUC12_df_T.iloc[0].astype(int).astype(str)  # setting 1st row as column name
westUS_HUC12_df_T = westUS_HUC12_df_T.iloc[1:]  # dropping 1st row from dataframe
westUS_HUC12_df_T = westUS_HUC12_df_T.reset_index()
westUS_HUC12_df_T.columns.values[0] = 'huc12'
westUS_HUC12_df_T.head()

Year,huc12,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,110300010201,0.005044,0.031210,0.044198,0.133766,0.049142,0.024015,0.000000,0.026800,0.000000,...,0.000000,0.053864,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000
1,170402150402,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000
2,170800050104,0.266710,0.332165,0.394226,0.254837,0.399572,0.322494,0.290203,0.302686,0.296448,...,0.279875,0.056480,0.267824,0.253682,0.13009,0.387836,0.392458,0.07482,0.132265,0.14954
3,170501070702,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000
4,181002040905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000


# Connecting the effective precip dataframe with the HUC12 shapefile

In [17]:
huc12_westUS_Peff = huc12_westUS.merge(westUS_HUC12_df_T, on='huc12')
huc12_westUS_Peff = huc12_westUS_Peff.drop(columns=['noncontr00', 'sourcedata',  'areaacres',  'humod', 'sourcefeat',
                                                                  'sourceorig', 'shape_area', 'shape_leng', 'hutype', 'tnmid', 'metasource',   
                                                                  'loaddate',    'gnis_id', 'noncontrib'])
huc12_westUS_Peff.head()

,huc12,tohuc,states,areasqkm,name,geometry,2000,2001,2002,2003,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,070200010408,070200011101,"MN,SD",293.39,Big Stone Lake,"POLYGON ((-96.83570 45.59745, -96.83561 45.598...",1.322797,1.572722,1.807652,1.80063,...,2.428478,2.031176,2.493218,2.713450,2.229837,2.522319,2.726200,2.936197,3.669061,2.992588
1,090201010203,090201010205,"MN,SD",102.27,Lower Lake Traverse,"POLYGON ((-96.74647 45.72157, -96.74638 45.721...",0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,090201010502,090201010505,"MN,ND,SD",147.33,Clubhouse Lake-Bois de Sioux River,"POLYGON ((-96.69140 45.93483, -96.69129 45.936...",0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.007012,0.006591,0.000000,0.006527,0.000000,0.000000,0.000000,0.000000
3,090201010505,090201010507,"MN,ND",111.84,County Ditch No 26-Bois de Sioux River,"POLYGON ((-96.67957 46.05348, -96.67957 46.053...",0.000000,0.000000,0.000000,0.00000,...,0.014116,0.044846,0.088185,0.015261,0.067617,0.022375,0.026419,0.000000,0.000000,0.000000
4,090201010507,090201040401,"MN,ND",146.65,Bois de Sioux River,"POLYGON ((-96.67903 46.18438, -96.67900 46.184...",0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.019849,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
huc12_westUS_Peff.columns

Index(['huc12', 'tohuc', 'states', 'areasqkm', 'name', 'geometry', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020'],
      dtype='object')

## removing HUC12s with no surface water use over 2000-2020

In [22]:
Peff_columns = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
                   '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
                   '2019', '2020']

In [23]:
huc12_westUS_Peff = huc12_westUS_Peff[(huc12_westUS_Peff[Peff_columns] > 0).any(axis=1)]
huc12_westUS_Peff.head()

,huc12,tohuc,states,areasqkm,name,geometry,2000,2001,2002,2003,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,070200010408,070200011101,"MN,SD",293.39,Big Stone Lake,"POLYGON ((-96.83570 45.59745, -96.83561 45.598...",1.322797,1.572722,1.807652,1.80063,...,2.428478,2.031176,2.493218,2.713450,2.229837,2.522319,2.726200,2.936197,3.669061,2.992588
2,090201010502,090201010505,"MN,ND,SD",147.33,Clubhouse Lake-Bois de Sioux River,"POLYGON ((-96.69140 45.93483, -96.69129 45.936...",0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.007012,0.006591,0.000000,0.006527,0.000000,0.000000,0.000000,0.000000
3,090201010505,090201010507,"MN,ND",111.84,County Ditch No 26-Bois de Sioux River,"POLYGON ((-96.67957 46.05348, -96.67957 46.053...",0.000000,0.000000,0.000000,0.00000,...,0.014116,0.044846,0.088185,0.015261,0.067617,0.022375,0.026419,0.000000,0.000000,0.000000
4,090201010507,090201040401,"MN,ND",146.65,Bois de Sioux River,"POLYGON ((-96.67903 46.18438, -96.67900 46.184...",0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.019849,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16,090201050103,090201050104,ND,161.92,Big Slough,"POLYGON ((-98.06522 46.21900, -98.06516 46.219...",9.457827,9.808426,9.947353,11.33536,...,13.802210,7.316948,8.156706,12.777290,5.443509,10.685040,11.315140,10.461750,13.256730,11.010460


## saving the HUC12 shapefiles with annual Peff

Peff unit in MGD-million gallon per day

In [24]:
huc12_westUS_Peff.to_file('../../Data_main/USGS_water_use_data/USGS_new_wateruse_data_HUC12/HUC12_WestUS_with_Annual_Peff.shp')